In [1]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder \
    .appName("KafkaReadExample") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0") \
    .getOrCreate()

25/01/05 13:23:08 WARN Utils: Your hostname, hadoop resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
25/01/05 13:23:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/usr/local/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/hadoop/.ivy2/cache
The jars for the packages stored in: /home/hadoop/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-088fafc6-4f8d-4de5-8186-cf69f9ee63c1;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.0 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
downloading https://repo1.maven.org/maven2/org/apache/spark/sp

In [2]:
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StringType, DoubleType, LongType

In [3]:
# Kafka configuration
kafka_brokers = "localhost:9092"  # Replace with your Kafka broker address
topic = "model-topic"

# Define the schema of the JSON data
schema = StructType() \
    .add("symbol", StringType()) \
    .add("timestamp", LongType()) \
    .add("source", StringType()) \
    .add("data_type", StringType()) \
    .add("bid", DoubleType()) \
    .add("ask", DoubleType()) \
    .add("price", DoubleType()) \
    .add("volume", DoubleType()) \
    .add("spread_raw", DoubleType()) \
    .add("spread_table", DoubleType()) \
    .add("volatility", DoubleType()) \
    .add("market_sentiment", DoubleType()) \
    .add("trading_activity", DoubleType())

In [4]:
# Read from Kafka
df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_brokers) \
    .option("subscribe", topic) \
    .option("startingOffsets", "earliest") \
    .option("failOnDataLoss", "false") \
    .load()

In [5]:
parsed_df = df.selectExpr("CAST(value AS STRING) as json") \
    .select(from_json(col("json"), schema).alias("data")) \
    .select("data.*")  # Flatten the nested structure

# Example: Filter and display rows where price is greater than 0
filtered_df = parsed_df.filter(col("price") > 0)

In [9]:
query = filtered_df.writeStream \
    .outputMode("append") \
    .format("console") \
    .option("truncate", "false") \
    .start()

query.awaitTermination()

25/01/05 13:27:28 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-f1f43028-402b-4c9a-ac67-cee587966858. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/01/05 13:27:28 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 5
-------------------------------------------
+------+-------------+----------+-----------+----+----+------+-----------+----------+------------+----------+----------------+----------------+
|symbol|timestamp    |source    |data_type  |bid |ask |price |volume     |spread_raw|spread_table|volatility|market_sentiment|trading_activity|
+------+-------------+----------+-----------+----+----+------+-----------+----------+------------+----------+----------------+----------------+
|XOM   |1736083644099|YLIFE_FEED|MARKET_DATA|-1.0|-1.0|112.03|1.2892229E7|-1.0      |-1.0        |0.56      |-0.318          |25.16           |
|SHEL  |1736083644761|YLIFE_FEED|MARKET_DATA|-1.0|-1.0|64.92 |4691606.0  |-1.0      |-1.0        |0.471     |0.53            |31.96           |
+------+-------------+----------+-----------+----+----+------+-----------+----------+------------+----------+----------------+----------------+



-------------------------------------------
Batch: 32
-------------------------------------------
+------+-------------+----------+-----------+----+----+-----+---------+----------+------------+----------+----------------+----------------+
|symbol|timestamp    |source    |data_type  |bid |ask |price|volume   |spread_raw|spread_table|volatility|market_sentiment|trading_activity|
+------+-------------+----------+-----------+----+----+-----+---------+----------+------------+----------+----------------+----------------+
|SHEL  |1736083644761|YLIFE_FEED|MARKET_DATA|-1.0|-1.0|64.92|4691606.0|-1.0      |-1.0        |0.471     |0.53            |31.96           |
|BP    |1736083646735|YLIFE_FEED|MARKET_DATA|-1.0|-1.0|31.09|6559706.0|-1.0      |-1.0        |0.61      |-0.94           |74.22           |
+------+-------------+----------+-----------+----+----+-----+---------+----------+------------+----------+----------------+----------------+



25/01/05 13:27:30 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


-------------------------------------------
Batch: 6
-------------------------------------------
+------+-------------+----------+-----------+----+----+------+-----------+----------+------------+----------+----------------+----------------+
|symbol|timestamp    |source    |data_type  |bid |ask |price |volume     |spread_raw|spread_table|volatility|market_sentiment|trading_activity|
+------+-------------+----------+-----------+----+----+------+-----------+----------+------------+----------+----------------+----------------+
|XOM   |1736083648481|YLIFE_FEED|MARKET_DATA|-1.0|-1.0|111.98|1.2627013E7|-1.0      |-1.0        |0.557     |-0.351          |26.83           |
|COP   |1736083647727|YLIFE_FEED|MARKET_DATA|-1.0|-1.0|103.22|3570777.0  |-1.0      |-1.0        |0.424     |0.402           |27.68           |
|BP    |1736083646735|YLIFE_FEED|MARKET_DATA|-1.0|-1.0|31.09 |6559706.0  |-1.0      |-1.0        |0.61      |-0.94           |74.22           |
+------+-------------+----------+------

...

-------------------------------------------
Batch: 14
-------------------------------------------
+------+-------------+----------+-----------+----+----+------+---------+----------+------------+----------+----------------+----------------+
|symbol|timestamp    |source    |data_type  |bid |ask |price |volume   |spread_raw|spread_table|volatility|market_sentiment|trading_activity|
+------+-------------+----------+-----------+----+----+------+---------+----------+------------+----------+----------------+----------------+
|COP   |1736083674778|YLIFE_FEED|MARKET_DATA|-1.0|-1.0|103.54|3980459.0|-1.0      |-1.0        |0.422     |0.647           |16.45           |
|BP    |1736083674583|YLIFE_FEED|MARKET_DATA|-1.0|-1.0|30.93 |6591958.0|-1.0      |-1.0        |0.626     |-0.94           |81.7            |
+------+-------------+----------+-----------+----+----+------+---------+----------+------------+----------+----------------+----------------+



-------------------------------------------
Batch: 8
-------------------------------------------
+------+-------------+----------+-----------+----+----+------+---------+----------+------------+----------+----------------+----------------+
|symbol|timestamp    |source    |data_type  |bid |ask |price |volume   |spread_raw|spread_table|volatility|market_sentiment|trading_activity|
+------+-------------+----------+-----------+----+----+------+---------+----------+------------+----------+----------------+----------------+
|COP   |1736083674778|YLIFE_FEED|MARKET_DATA|-1.0|-1.0|103.54|3980459.0|-1.0      |-1.0        |0.422     |0.647           |16.45           |
|BP    |1736083674583|YLIFE_FEED|MARKET_DATA|-1.0|-1.0|30.93 |6591958.0|-1.0      |-1.0        |0.626     |-0.94           |81.7            |
+------+-------------+----------+-----------+----+----+------+---------+----------+------------+----------+----------------+----------------+

-------------------------------------------
Batch:

-------------------------------------------
Batch: 15
-------------------------------------------
+------+-------------+----------+-----------+----+----+-----+---------+----------+------------+----------+----------------+----------------+
|symbol|timestamp    |source    |data_type  |bid |ask |price|volume   |spread_raw|spread_table|volatility|market_sentiment|trading_activity|
+------+-------------+----------+-----------+----+----+-----+---------+----------+------------+----------+----------------+----------------+
|SHEL  |1736083676778|YLIFE_FEED|MARKET_DATA|-1.0|-1.0|64.56|4436033.0|-1.0      |-1.0        |0.473     |0.394           |32.81           |
|BP    |1736083677468|YLIFE_FEED|MARKET_DATA|-1.0|-1.0|30.86|6657648.0|-1.0      |-1.0        |0.629     |-0.924          |86.01           |
+------+-------------+----------+-----------+----+----+-----+---------+----------+------------+----------+----------------+----------------+



-------------------------------------------
Batch: 9
-------------------------------------------
+------+-------------+----------+-----------+----+----+-----+---------+----------+------------+----------+----------------+----------------+
|symbol|timestamp    |source    |data_type  |bid |ask |price|volume   |spread_raw|spread_table|volatility|market_sentiment|trading_activity|
+------+-------------+----------+-----------+----+----+-----+---------+----------+------------+----------+----------------+----------------+
|SHEL  |1736083676778|YLIFE_FEED|MARKET_DATA|-1.0|-1.0|64.56|4436033.0|-1.0      |-1.0        |0.473     |0.394           |32.81           |
|BP    |1736083677468|YLIFE_FEED|MARKET_DATA|-1.0|-1.0|30.86|6657648.0|-1.0      |-1.0        |0.629     |-0.924          |86.01           |
+------+-------------+----------+-----------+----+----+-----+---------+----------+------------+----------+----------------+----------------+

-------------------------------------------
Batch: 42
--

-------------------------------------------
Batch: 16
-------------------------------------------
+------+-------------+----------+-----------+----+----+-----+---------+----------+------------+----------+----------------+----------------+
|symbol|timestamp    |source    |data_type  |bid |ask |price|volume   |spread_raw|spread_table|volatility|market_sentiment|trading_activity|
+------+-------------+----------+-----------+----+----+-----+---------+----------+------------+----------+----------------+----------------+
|BP    |1736083679114|YLIFE_FEED|MARKET_DATA|-1.0|-1.0|30.9 |6731741.0|-1.0      |-1.0        |0.619     |-0.847          |85.73           |
+------+-------------+----------+-----------+----+----+-----+---------+----------+------------+----------+----------------+----------------+



In [10]:
query.stop()


25/01/05 13:28:03 ERROR WriteToDataSourceV2Exec: Data source write support MicroBatchWrite[epoch: 10, writer: ConsoleWriter[numRows=20, truncate=false]] is aborting.
25/01/05 13:28:03 ERROR WriteToDataSourceV2Exec: Data source write support MicroBatchWrite[epoch: 10, writer: ConsoleWriter[numRows=20, truncate=false]] aborted.


-------------------------------------------
Batch: 17
-------------------------------------------
-------------------------------------------
Batch: 43
-------------------------------------------
+------+-------------+----------+-----------+----+----+-----+---------+----------+------------+----------+----------------+----------------+
|symbol|timestamp    |source    |data_type  |bid |ask |price|volume   |spread_raw|spread_table|volatility|market_sentiment|trading_activity|
+------+-------------+----------+-----------+----+----+-----+---------+----------+------------+----------+----------------+----------------+
|BP    |1736083679114|YLIFE_FEED|MARKET_DATA|-1.0|-1.0|30.9 |6731741.0|-1.0      |-1.0        |0.619     |-0.847          |85.73           |
|BP    |1736083680232|YLIFE_FEED|MARKET_DATA|-1.0|-1.0|30.75|6569666.0|-1.0      |-1.0        |0.607     |-0.911          |88.34           |
+------+-------------+----------+-----------+----+----+-----+---------+----------+------------+----

+------+-------------+----------+-----------+----+----+-----+---------+----------+------------+----------+----------------+----------------+
|symbol|timestamp    |source    |data_type  |bid |ask |price|volume   |spread_raw|spread_table|volatility|market_sentiment|trading_activity|
+------+-------------+----------+-----------+----+----+-----+---------+----------+------------+----------+----------------+----------------+
|BP    |1736083679114|YLIFE_FEED|MARKET_DATA|-1.0|-1.0|30.9 |6731741.0|-1.0      |-1.0        |0.619     |-0.847          |85.73           |
|BP    |1736083680232|YLIFE_FEED|MARKET_DATA|-1.0|-1.0|30.75|6569666.0|-1.0      |-1.0        |0.607     |-0.911          |88.34           |
+------+-------------+----------+-----------+----+----+-----+---------+----------+------------+----------+----------------+----------------+



-------------------------------------------
Batch: 17
-------------------------------------------
+------+-------------+----------+-----------+----+----+------+---------+----------+------------+----------+----------------+----------------+
|symbol|timestamp    |source    |data_type  |bid |ask |price |volume   |spread_raw|spread_table|volatility|market_sentiment|trading_activity|
+------+-------------+----------+-----------+----+----+------+---------+----------+------------+----------+----------------+----------------+
|COP   |1736083682235|YLIFE_FEED|MARKET_DATA|-1.0|-1.0|103.81|4152847.0|-1.0      |-1.0        |0.429     |0.722           |15.49           |
|BP    |1736083680232|YLIFE_FEED|MARKET_DATA|-1.0|-1.0|30.75 |6569666.0|-1.0      |-1.0        |0.607     |-0.911          |88.34           |
|SHEL  |1736083681982|YLIFE_FEED|MARKET_DATA|-1.0|-1.0|64.55 |4458097.0|-1.0      |-1.0        |0.476     |0.471           |35.74           |
+------+-------------+----------+-----------+----+

-------------------------------------------
Batch: 18
-------------------------------------------
+------+-------------+----------+-----------+----+----+------+---------+----------+------------+----------+----------------+----------------+
|symbol|timestamp    |source    |data_type  |bid |ask |price |volume   |spread_raw|spread_table|volatility|market_sentiment|trading_activity|
+------+-------------+----------+-----------+----+----+------+---------+----------+------------+----------+----------------+----------------+
|COP   |1736083682235|YLIFE_FEED|MARKET_DATA|-1.0|-1.0|103.81|4152847.0|-1.0      |-1.0        |0.429     |0.722           |15.49           |
|SHEL  |1736083684030|YLIFE_FEED|MARKET_DATA|-1.0|-1.0|64.38 |4349685.0|-1.0      |-1.0        |0.474     |0.464           |31.07           |
|SHEL  |1736083681982|YLIFE_FEED|MARKET_DATA|-1.0|-1.0|64.55 |4458097.0|-1.0      |-1.0        |0.476     |0.471           |35.74           |
+------+-------------+----------+-----------+----+

...

-------------------------------------------
Batch: 26
-------------------------------------------
-------------------------------------------
Batch: 52
-------------------------------------------
+------+-------------+----------+-----------+----+----+------+-----------+----------+------------+----------+----------------+----------------+
|symbol|timestamp    |source    |data_type  |bid |ask |price |volume     |spread_raw|spread_table|volatility|market_sentiment|trading_activity|
+------+-------------+----------+-----------+----+----+------+-----------+----------+------------+----------+----------------+----------------+
|XOM   |1736083700071|YLIFE_FEED|MARKET_DATA|-1.0|-1.0|112.93|1.2107616E7|-1.0      |-1.0        |0.541     |-0.434          |19.49           |
|XOM   |1736083699213|YLIFE_FEED|MARKET_DATA|-1.0|-1.0|112.45|1.2230219E7|-1.0      |-1.0        |0.549     |-0.435          |22.89           |
+------+-------------+----------+-----------+----+----+------+-----------+----------

-------------------------------------------
Batch: 27
-------------------------------------------
+------+-------------+----------+-----------+----+----+------+-----------+----------+------------+----------+----------------+----------------+
|symbol|timestamp    |source    |data_type  |bid |ask |price |volume     |spread_raw|spread_table|volatility|market_sentiment|trading_activity|
+------+-------------+----------+-----------+----+----+------+-----------+----------+------------+----------+----------------+----------------+
|SHEL  |1736083703315|YLIFE_FEED|MARKET_DATA|-1.0|-1.0|64.13 |3884693.0  |-1.0      |-1.0        |0.483     |0.757           |40.89           |
|XOM   |1736083703766|YLIFE_FEED|MARKET_DATA|-1.0|-1.0|112.51|1.1709951E7|-1.0      |-1.0        |0.534     |-0.42           |24.15           |
+------+-------------+----------+-----------+----+----+------+-----------+----------+------------+----------+----------------+----------------+

-------------------------------------

-------------------------------------------
Batch: 28
-------------------------------------------
+------+-------------+----------+-----------+----+----+------+-----------+----------+------------+----------+----------------+----------------+
|symbol|timestamp    |source    |data_type  |bid |ask |price |volume     |spread_raw|spread_table|volatility|market_sentiment|trading_activity|
+------+-------------+----------+-----------+----+----+------+-----------+----------+------------+----------+----------------+----------------+
|XOM   |1736083705426|YLIFE_FEED|MARKET_DATA|-1.0|-1.0|112.62|1.093833E7 |-1.0      |-1.0        |0.535     |-0.444          |24.82           |
|XOM   |1736083704638|YLIFE_FEED|MARKET_DATA|-1.0|-1.0|113.03|1.1253232E7|-1.0      |-1.0        |0.539     |-0.446          |25.4            |
+------+-------------+----------+-----------+----+----+------+-----------+----------+------------+----------+----------------+----------------+



-------------------------------------------
Batch: 29
-------------------------------------------
+------+-------------+----------+-----------+----+----+------+----------+----------+------------+----------+----------------+----------------+
|symbol|timestamp    |source    |data_type  |bid |ask |price |volume    |spread_raw|spread_table|volatility|market_sentiment|trading_activity|
+------+-------------+----------+-----------+----+----+------+----------+----------+------------+----------+----------------+----------------+
|XOM   |1736083705426|YLIFE_FEED|MARKET_DATA|-1.0|-1.0|112.62|1.093833E7|-1.0      |-1.0        |0.535     |-0.444          |24.82           |
+------+-------------+----------+-----------+----+----+------+----------+----------+------------+----------+----------------+----------------+

-------------------------------------------
Batch: 55
-------------------------------------------
+------+-------------+----------+-----------+----+----+------+----------+----------+----

-------------------------------------------
Batch: 29
-------------------------------------------
+------+-------------+----------+-----------+----+----+-----+---------+----------+------------+----------+----------------+----------------+
|symbol|timestamp    |source    |data_type  |bid |ask |price|volume   |spread_raw|spread_table|volatility|market_sentiment|trading_activity|
+------+-------------+----------+-----------+----+----+-----+---------+----------+------------+----------+----------------+----------------+
|BP    |1736083707223|YLIFE_FEED|MARKET_DATA|-1.0|-1.0|30.47|6905611.0|-1.0      |-1.0        |0.597     |-0.938          |78.83           |
+------+-------------+----------+-----------+----+----+-----+---------+----------+------------+----------+----------------+----------------+



In [11]:
for active_query in spark.streams.active:
    active_query.stop()


25/01/05 13:28:30 WARN Shell: Interrupted while joining on: Thread[Thread-2701,5,]
java.lang.InterruptedException
	at java.base/java.lang.Object.wait(Native Method)
	at java.base/java.lang.Thread.join(Thread.java:1300)
	at java.base/java.lang.Thread.join(Thread.java:1375)
	at org.apache.hadoop.util.Shell.joinThread(Shell.java:1042)
	at org.apache.hadoop.util.Shell.runCommand(Shell.java:1002)
	at org.apache.hadoop.util.Shell.run(Shell.java:900)
	at org.apache.hadoop.util.Shell$ShellCommandExecutor.execute(Shell.java:1212)
	at org.apache.hadoop.util.Shell.execCommand(Shell.java:1306)
	at org.apache.hadoop.util.Shell.execCommand(Shell.java:1288)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem$LocalFSFileOutputStream.<init>(RawLocalFileSystem.java:324)
	at org.apache.hadoop.fs.RawLocalFileSystem$LocalFSFileOutputStream.<init>(RawLocalFileSystem.java:294)
	at org.apache.hadoop.fs.RawLocalFileSystem.createOutp

-------------------------------------------
Batch: 30
-------------------------------------------
+------+-------------+----------+-----------+----+----+-----+---------+----------+------------+----------+----------------+----------------+
|symbol|timestamp    |source    |data_type  |bid |ask |price|volume   |spread_raw|spread_table|volatility|market_sentiment|trading_activity|
+------+-------------+----------+-----------+----+----+-----+---------+----------+------------+----------+----------------+----------------+
|SHEL  |1736083709032|YLIFE_FEED|MARKET_DATA|-1.0|-1.0|63.91|3868376.0|-1.0      |-1.0        |0.489     |0.754           |39.31           |
|BP    |1736083707223|YLIFE_FEED|MARKET_DATA|-1.0|-1.0|30.47|6905611.0|-1.0      |-1.0        |0.597     |-0.938          |78.83           |
+------+-------------+----------+-----------+----+----+-----+---------+----------+------------+----------+----------------+----------------+

-------------------------------------------
Batch: 56
-

-------------------------------------------
Batch: 30
-------------------------------------------
+------+-------------+----------+-----------+----+----+------+---------+----------+------------+----------+----------------+----------------+
|symbol|timestamp    |source    |data_type  |bid |ask |price |volume   |spread_raw|spread_table|volatility|market_sentiment|trading_activity|
+------+-------------+----------+-----------+----+----+------+---------+----------+------------+----------+----------------+----------------+
|SHEL  |1736083709032|YLIFE_FEED|MARKET_DATA|-1.0|-1.0|63.91 |3868376.0|-1.0      |-1.0        |0.489     |0.754           |39.31           |
|COP   |1736083710416|YLIFE_FEED|MARKET_DATA|-1.0|-1.0|103.17|4131844.0|-1.0      |-1.0        |0.422     |0.794           |20.85           |
|COP   |1736083710032|YLIFE_FEED|MARKET_DATA|-1.0|-1.0|103.44|4211020.0|-1.0      |-1.0        |0.425     |0.837           |16.88           |
+------+-------------+----------+-----------+----+

25/01/05 13:28:32 ERROR WriteToDataSourceV2Exec: Data source write support MicroBatchWrite[epoch: 57, writer: ConsoleWriter[numRows=20, truncate=false]] is aborting.
25/01/05 13:28:32 ERROR WriteToDataSourceV2Exec: Data source write support MicroBatchWrite[epoch: 57, writer: ConsoleWriter[numRows=20, truncate=false]] aborted.
25/01/05 13:28:32 ERROR WriteToDataSourceV2Exec: Data source write support MicroBatchWrite[epoch: 31, writer: ConsoleWriter[numRows=20, truncate=false]] is aborting.
25/01/05 13:28:32 ERROR WriteToDataSourceV2Exec: Data source write support MicroBatchWrite[epoch: 31, writer: ConsoleWriter[numRows=20, truncate=false]] aborted.
25/01/05 13:28:32 ERROR Utils: Aborting task
org.apache.spark.TaskKilledException
	at org.apache.spark.TaskContextImpl.killTaskIfInterrupted(TaskContextImpl.scala:267)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:36)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at scala.collection.Itera

25/01/05 13:28:32 ERROR Utils: Aborting task
org.apache.spark.TaskKilledException
	at org.apache.spark.TaskContextImpl.killTaskIfInterrupted(TaskContextImpl.scala:267)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:36)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at scala.collection.Iterator$$anon$12.hasNext(Iterator.scala:513)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenEvaluatorFactory$WholeStageCodegenPartitionEvaluator$$anon$1.hasNext(WholeStageCodegenEvaluatorFactory.scala:43)
	at org.apache.spark.sql.execution.datasources.v2.WritingSparkTask.$anonfun$run$5(WriteToDataSourceV2Exec.scala:446)
	at org.apache.spark.util.Utils$.tryWithSafeFinally

In [ ]:
# Kafka configuration
kafka_brokers = "localhost:9092"  # Replace with your Kafka broker address
topic = "model-topic"

# Read from Kafka
df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_brokers) \
    .option("subscribe", topic) \
    .option("startingOffsets", "earliest") \
    .option("failOnDataLoss", "false") \
    .load()

# Select the key and value from Kafka messages
df = df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

# Start the streaming query to console for testing
query = df.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

query.awaitTermination()